## <font color='darkblue'>Preface</font>
([article source](https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/)) <font size='3ptx'>**[Keras](https://keras.io/) is one of the most popular deep learning libraries in Python for research and development because of its simplicity and ease of use. The [scikit-learn library](https://scikit-learn.org/stable/) is the most popular library for general machine learning in Python.**</font>

**In this post you will discover how you can use deep learning models from Keras with the scikit-learn library in Python**. This will allow you to leverage the power of the scikit-learn library for tasks like model evaluation and model hyper-parameter optimization.

In [12]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

### <font color='darkgreen'>Overview</font>
**[Keras](https://keras.io/) is a popular library for deep learning in Python**, but the focus of the library is deep learning. In fact it strives for minimalism, focusing on only what you need to quickly and simply define and build deep learning models.

The scikit-learn library in Python is built upon the SciPy stack for efficient numerical computation. It is a fully featured library for general machine learning and provides many utilities that are useful in the development of deep learning models. Not least:
* Evaluation of models using resampling methods like k-fold cross validation.
* Efficient search and evaluation of model hyper-parameters.

The Keras library provides a [convenient wrapper for deep learning models to be used as classification or regression estimators in scikit-learn](https://faroit.com/keras-docs/1.0.6/scikit-learn-api/). In the next sections, we will work through examples of using the KerasClassifier wrapper for a classification neural network created in Keras and used in the scikit-learn library.

### <font color='darkgreen'>Data Set</font>
The test problem is the [Pima Indians onset of diabetes classification dataset](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes). This is a small dataset with all numerical attributes that is easy to work with. [Download the dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data) and place it in your currently working directly with the name <font color='olive'>pima-indians-diabetes.csv</font> (<font color='brown'>update:</font> [download from here](https://www.kaggle.com/uciml/pima-indians-diabetes-database)).

In [9]:
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
target_columns = ['Outcome']
pima_df = pd.read_csv('../../datas/kaggle_pima-indians-diabetes-database/diabetes.csv')
pima_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## <font color='darkblue'>Evaluate Deep Learning Models with Cross Validation</font>
The <font color='darkblue'>**KerasClassifier**</font> and <font color='darkblue'>**KerasRegressor**</font> classes in Keras take an **argument <font color='violet'>build_fn</font> which is the name of the function to call to get your model.** You must define a function called whatever you like that defines your model, compiles it and returns it.

In the example, below we define a function <font color='blue'>create_model()</font> that create a simple multi-layer neural network for the problem.

We pass this function name to the <font color='darkblue'>**KerasClassifier**</font> class by the <font color='violet'>build_fn</font> argument. We also pass in additional arguments of <font color='blue'>nb_epoch=150</font> and <font color='blue'>batch_size=10</font>. These are automatically bundled up and passed on to the <font color='blue'>fit()</font> function which is called internally by the <font color='darkblue'>**KerasClassifier**</font> class.

In this example, we use the scikit-learn [**StratifiedKFold**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) to perform 10-fold stratified cross-validation. This is a resampling technique that can provide a robust estimate of the performance of a machine learning model on unseen data.

We use the scikit-learn function [cross_val_score()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to evaluate our model using the cross-validation scheme and print the results.

In [10]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# split into input (X) and output (Y) variables
X = pima_df[feature_columns]
Y = pima_df[target_columns]
# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.7319207131862641


Running the example displays the skill of the model for each epoch. A total of 10 models are created and evaluated and the final average accuracy is displayed (73.2%).

## <font color='darkblue'>Grid Search Deep Learning Model Parameters</font>
**The previous example showed how easy it is to wrap your deep learning model from Keras and use it in functions from the scikit-learn library.**

In this example, we go a step further. The function that we specify to the <font color='violet'>build_fn</font> argument when creating the <font color='darkblue'>**KerasClassifier**</font> wrapper can take arguments. We can use these arguments to further customize the construction of the model. In addition, we know we can provide arguments to the <font color='blue'>fit()</font> function.

**In this example, we use a grid search to evaluate different configurations for our neural network model and report on the combination that provides the best-estimated performance.**

The <font color='blue'>create_model()</font> function is defined to take two arguments <font color='violet'>optimizer</font> and <font color='violet'>init</font>, both of which must have default values. This will allow us to evaluate the effect of using different optimization algorithms and weight initialization schemes for our network.

After creating our model, we define arrays of values for the parameter we wish to search, specifically:
* Optimizers for searching different weight values.
* Initializers for preparing the network weights using different schemes.
* Epochs for training the model for a different number of exposures to the training dataset.
* Batches for varying the number of samples before a weight update.

The options are specified into a dictionary and passed to the configuration of the [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) scikit-learn class. This class will evaluate a version of our neural network model for each combination of parameters (<font color='brown'>2 x 3 x 3 x 3 for the combinations of optimizers, initializations, epochs and batches</font>). Each combination is then evaluated using the default of 3-fold stratified cross validation.

**That is a lot of models and a lot of computation. This is not a scheme that you want to use lightly because of the time it will take. It may be useful for you to design small experiments with a smaller subset of your data that will complete in a reasonable time**. This is reasonable in this case because of the small network and the small dataset (<font color='brown'>less than 1000 instances and 9 attributes</font>).

Finally, the performance and combination of configurations for the best model are displayed, followed by the performance of all combinations of parameters.

In [13]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(
    optimizer=optimizers,
    epochs=epochs,
    batch_size=batches, 
    init=init
)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.764383 using {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'adam'}
0.670631 (0.057422) with: {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.707088 (0.037163) with: {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.735727 (0.028829) with: {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.714871 (0.026510) with: {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'adam'}
0.724030 (0.032225) with: {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.704516 (0.039423) with: {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'adam'}
0.699329 (0.056497) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.701850 (0.025562) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.739640 (0.035691) with: {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optim

This might take about 5 minutes to complete on your workstation executed on the CPU (<font color='brown'>rather than CPU</font>). running the example shows the results below.

We can see that the grid search discovered that using a uniform initialization scheme, rmsprop optimizer, 150 epochs and a batch size of 5 achieved the best cross-validation score of approximately 75% on this problem.

For a fuller example of tuning hyperparameters with Keras, see the tutorial:
* [How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)